In [1]:
!pip install convokit
# !pip install transformers[torch]
# !pip install accelerate
# !pip install transformers_interpret
# !pip install git+https://github.com/allenai/longformer.git
!pip install openai==0.28
!pip install jsonlines
!pip install shap
!pip install pdpbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.7 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=28c8555f829343f0f2c1b325ebb46e225b99d00bb83497258c9f984356dbb5be
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c33e0d5525b231
 

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
import matplotlib.pyplot as plt
import numpy as np
import re
import openai
import time
import pandas as pd
import jsonlines
import csv
import json
import random
import torch
import convokit
import spacy

from sklearn.datasets import fetch_openml
from sklearn import tree
from datetime import date
from torch.nn import L1Loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from scipy import stats
from convokit import Corpus, Speaker, Utterance
from convokit import download
from convokit import TextParser

openai.api_key = "XXXXXXXX"

Mounted at /content/drive


# 0. Load Data Function

## 0.1. OUM data

In [3]:
# Load the data
def load_data_oum(label='after'):
    final_convs = []
    final_labels = []
    wizards_data = []
    moral_foundations = ["care", "fairness", "liberty", "loyalty", "authority", "sanctity", "none"]
    input_files = {"wizards": "wizards_dialogues.json", "final_argubot": "argubot_final_exp.json",
                   "models_dialogues": "models_dialogues.json"}
    dials_with_scores = {"wizards": {}, "final_argubot": {}, "models_dialogues": {}}

    for key in input_files:
        input_file = input_files[key]
        with open(input_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for d in data:
            is_wiki = False
            for m in d["messages"]:
                if 'model' in m and (m['model'] == 'wikibot' or m['model'] == 'controlbot'):
                    is_wiki = True
                    break
            if is_wiki:
                continue
            yes_no = 'none'
            k = 'Did you vote for (Leave) or against (Remain) Brexit in the 2016 UK referendum?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'against (remain)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'for (leave)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'

            k = 'In the referendum on whether the UK should remain a member of the EU (BREXIT), how did you vote?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'remain (against brexit)':
                    yes_no = 'no'
                elif d['participant_info'][k].lower() == 'leave (for brexit)':
                    yes_no = 'yes'
                else:
                    yes_no = 'none'
            k = 'Have you had at least one dose of an approved Covid-19 vaccine?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'
            k = 'Are you a vegan?'
            if k in d['participant_info']:
                if d['participant_info'][k].lower() == 'yes':
                    yes_no = 'yes'
                elif d['participant_info'][k].lower() == 'no':
                    yes_no = 'no'

            if yes_no == 'none':
                continue

            if 'Questions' in d['participant_info']:
                for q in d['participant_info']['Questions']:
                    if "final" in input_file:
                        if label == 'oum':
                            continue
                        if d['participant_info']['Questions'][q]['after'] == -1:
                            continue
                    elif d['participant_info']['Questions'][q]['before'] == -1 or d['participant_info']['Questions'][q]['after'] == -1:
                        continue
                    if 'good reasons' in q.lower():
                        if d['topic'] != 'brexit' and 'not' in q.lower() and yes_no == 'no':
                            continue
                        if d['topic'] != 'brexit' and 'not' not in q.lower() and yes_no == 'yes':
                            continue
                        if 'leave' in q.lower() and yes_no == 'yes':
                            continue
                        if 'remain' in q.lower() and yes_no == 'no':
                            continue
                        if d["_id"] not in dials_with_scores[key]:
                            text = ''
                            dials_with_scores[key][d["_id"]] = {"topic": d["topic"], "dataset": key}
                            for message in d['messages']:
                                if message['role'] == 'admin' or 'modified_argument' not in message:
                                    continue

                                text = text + '\n\n' + '<' + message['role'] + '>' + '\n' + message['modified_argument']
                            dials_with_scores[key][d["_id"]]['text'] = text.strip()
                            final_convs.append(text.strip())

                    if 'good reasons' in q.lower():
                        if False and label == 'oum': ############## False to ensure that we always calcuclate the OUM after the conversation
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']) - float(d['participant_info']['Questions'][q]['before']))
                        else:
                            final_labels.append(float(d['participant_info']['Questions'][q]['after']))
                        oum = d['participant_info']['Questions'][q]['after'] - d['participant_info']['Questions'][q]['before'] if "final" not in input_file else None
                        dials_with_scores[key][d["_id"]]["good_reasons"] = {"oum": oum, "after": d['participant_info']['Questions'][q]['after']}
                        if 'before' in d['participant_info']['Questions'][q] and d['participant_info']['Questions'][q]['before'] != -1:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = d['participant_info']['Questions'][q]['before']
                        else:
                            dials_with_scores[key][d["_id"]]["good_reasons"]['before'] = None

    assert len(final_convs) == len(final_labels)
    return final_convs, final_labels



def get_utterances_oum(text):
    # Splitting the input text into lines
    lines = text.split('\n')
    # Variable to keep the cleaned lines
    cleaned_lines = []
    # Variable to keep track of whether the next line should be added
    add_next_line = False
    for line in lines:
        # If the line is a participant tag, set the flag to add the next line
        if line.strip() == '<participant>':
            add_next_line = True
        elif line.strip() in ['<woz>', '<chatbot>']:
            add_next_line = True
        elif add_next_line:
            # If the flag is set, add this line to the cleaned list and reset the flag
            cleaned_lines.append(line)
            add_next_line = False
    # Join the cleaned lines back into a single string
    cleaned_text = '\n\n'.join(cleaned_lines)
    return cleaned_text

### Bot/Woz's data
conversations, labels = load_data_oum(label='after')
utterances = [get_utterances_oum(c) for c in conversations]


## 0.2. Wikitactics dataset

In [4]:
import json
import pandas as pd
import numpy as np
from collections import Counter

def load_data_wikitac():
    with open('./wikitactics.json') as f:
        data = json.load(f)

    conversations = []
    utterances_cleaned = []
    labels = []
    # Extract conversations/disputes for ESCALATED disputes
    for dispute in data:
        users = list()
        conversation = ''
        utt_cleaned = ''
        for utterance in dispute['utterances']:
            username = utterance['username']
            text = utterance['text']
            conversation += f"<user_id={username}>\n{text}\n\n"
            utt_cleaned += text + '\n\n'
        conversations.append(conversation)
        utterances_cleaned.append(utt_cleaned)
        labels.append(dispute['escalation_label'])

    return conversations, utterances_cleaned, labels

conversations, utterances, labels = load_data_wikitac()

## 0.3. Wikipedia for Deletion

In [7]:
import collections
import random
import json
import pandas as pd
import numpy as np
from collections import Counter
from convokit import Corpus, download

def load_data_afd():
    # Load the data from the JSON file
    with open('afd_1000_randomised_dialogues.json', 'r') as json_file:
        data_dict = json.load(json_file)

    # Extract the conversations, utterances, and labels from the data dictionary
    conversations = data_dict['conversations']
    utterances = data_dict['utterances']
    labels = data_dict['labels']
    labels = [1 if i == 0 else 0 for i in labels]
    return conversations, utterances, labels

conversations, utterances, labels = load_data_afd()
labels

# 1. Reading Data & Training Models


## 1.1. Loading and parsing dataset

### 1.1.1. OUM dataset

In [ ]:
conversations, labels = load_data_oum(label='after')
utterances = [get_utterances_oum(c) for c in conversations]

### 1.1.2. Wikitactics dataset

In [ ]:
conversations, utterances, labels = load_data_wikitac()

### 1.1.3. Wikipedia for Deletion

In [9]:
conversations, utterances, labels = load_data_afd()

## 1.2. Politeness Markers

### 1.2.0. Explaining the features

In [ ]:
"""
Please:

Checks if the word "please" appears anywhere in the sentence.

Start with 'Please':

Checks if the sentence starts with the word "please".

Has Subject Hedge:

Identifies if any subject in the sentence depends on a hedge word.

Use of 'by the way':

Checks if the phrase "by the way" is used in the sentence.

Hedge words:

Identifies if any word in the sentence is a hedge word.

Assert factuality:

Checks for words that assert factuality, like "in fact," "actually," or "really".

Start with Deference:

Checks if the sentence starts with deferential words like "great," "good," or "nice".

Gratitude:

Identifies expressions of gratitude, like "thank" or "thanks".

Apologising:

Identifies apologetic expressions like "sorry," "oops," or "I apologize".

1st person plural:

Checks for first-person plural pronouns like "we," "our," or "us".

1st person pronouns:

Identifies first-person singular pronouns like "I," "my," or "mine".

Start with 1st person:

Checks if the sentence starts with a first-person singular pronoun.

2nd person pronouns:

Identifies second-person pronouns like "you," "your," or "yours".

Start with 2nd person:

Checks if the sentence starts with a second-person pronoun.

Start with greeting:

Checks if the sentence starts with a greeting like "hi," "hello," or "hey".

Starts with Question:

Identifies if the sentence starts with a question word like "what," "why," "who," or "how".

Starts with Conjunction:

Checks if the sentence starts with a conjunction or transition word like "so," "then," "and," "but," or "or".

Positive words:

Identifies the presence of positive sentiment words.

Negative words:

Identifies the presence of negative sentiment words.

Subjunctive words:

Checks for the use of subjunctive mood words like "could" or "would" when preceded by "you".

Indicative words:

Checks for the use of indicative mood words like "can" or "will" when preceded by "you".
"""

'\nPlease:\n\nChecks if the word "please" appears anywhere in the sentence.\n\nStart with \'Please\':\n\nChecks if the sentence starts with the word "please".\n\nHas Subject Hedge:\n\nIdentifies if any subject in the sentence depends on a hedge word.\n\nUse of \'by the way\':\n\nChecks if the phrase "by the way" is used in the sentence.\n\nHedge words:\n\nIdentifies if any word in the sentence is a hedge word.\n\nAssert factuality:\n\nChecks for words that assert factuality, like "in fact," "actually," or "really".\n\nStart with Deference:\n\nChecks if the sentence starts with deferential words like "great," "good," or "nice".\n\nGratitude:\n\nIdentifies expressions of gratitude, like "thank" or "thanks".\n\nApologising:\n\nIdentifies apologetic expressions like "sorry," "oops," or "I apologize".\n\n1st person plural:\n\nChecks for first-person plural pronouns like "we," "our," or "us".\n\n1st person pronouns:\n\nIdentifies first-person singular pronouns like "I," "my," or "mine".\n\nS

In [ ]:
"""
# words: This represents the total number of words in a message. It is calculated by counting the number of tokens (words) in the message.

# me pronoun: This feature counts the number of first-person singular pronouns (e.g., "I", "me", "my") in a message. These pronouns are specified in the pron_me list of the lexicons dictionary.

# we pronoun: This feature counts the number of first-person plural pronouns (e.g., "we", "us", "our") in a message. These pronouns are specified in the pron_we list of the lexicons dictionary.

# you pronoun: This feature counts the number of second-person pronouns (e.g., "you", "your") in a message. These pronouns are specified in the pron_you list of the lexicons dictionary.

# 3rd person pronouns: This feature counts the number of third-person pronouns (e.g., "he", "she", "they") in a message. These pronouns are specified in the pron_3rd list of the lexicons dictionary.

# Geography terms: This feature counts the number of geography-related terms in a message. These terms are specified in the geo list, which is loaded from the my_geo.txt file in the lexicons directory.

# Meta terms: This feature counts the number of meta-discourse terms in a message. These terms are specified in the meta list, which is loaded from the my_meta.txt file in the lexicons directory.
# Meta terms: Terms are associated with the functionalities, actions, and elements within the StreetCrowd environment

# Certainty terms: This feature counts the number of words expressing certainty in a message. These terms are specified in the certain list, which is loaded from the my_certain.txt file in the lexicons directory.

# Hedging terms: This feature counts the number of hedging terms (words that indicate uncertainty) in a message. These terms are specified in the hedge list, which is loaded from the my_hedges.txt file in the lexicons directory.

# New content words: This represents the number of new content words introduced by a user in a message. New content words are content words (significant words such as nouns, verbs, adjectives) that have not been seen in any previous messages. They exclude common stopwords and are identified based on part-of-speech tags.

# New content words * # Certainty terms: This represents the number of new content words introduced in a message that are also accompanied by certainty terms. It is calculated as the product of the number of new content words (# New content words) and the count of certainty terms (# Certainty terms) in the message.

# New content words * # Hedging terms: This represents the number of new content words introduced in a message that are also accompanied by hedging terms. It is calculated as the product of the number of new content words (# New content words) and the count of hedging terms (# Hedging terms) in the message.

These features are designed to capture various aspects of the language used in messages, including the use of personal pronouns, specific types of terms, and the introduction of new content in the conversation. The multiplication features specifically look at how new content words interact with expressions of certainty and hedging within the messages.
"""

'\n# words: This represents the total number of words in a message. It is calculated by counting the number of tokens (words) in the message.\n\n# me pronoun: This feature counts the number of first-person singular pronouns (e.g., "I", "me", "my") in a message. These pronouns are specified in the pron_me list of the lexicons dictionary.\n\n# we pronoun: This feature counts the number of first-person plural pronouns (e.g., "we", "us", "our") in a message. These pronouns are specified in the pron_we list of the lexicons dictionary.\n\n# you pronoun: This feature counts the number of second-person pronouns (e.g., "you", "your") in a message. These pronouns are specified in the pron_you list of the lexicons dictionary.\n\n# 3rd person pronouns: This feature counts the number of third-person pronouns (e.g., "he", "she", "they") in a message. These pronouns are specified in the pron_3rd list of the lexicons dictionary.\n\n# Geography terms: This feature counts the number of geography-related

### 1.2.1. Generating Politeness Markers

In [21]:
from convokit import PolitenessStrategies
from collections import defaultdict
import numpy as np

if len(conversations) != 213 and len(conversations) != 542 and len(conversations) != 1000:
    ps = PolitenessStrategies()
    spacy_nlp = spacy.load('en_core_web_sm', disable=['ner'])

    poli_features_mean = defaultdict(list)
    poli_features_grad = defaultdict(list)

    for i, c in enumerate(utterances):
        if i%100==0:
            print(i)
        poli_evolution = defaultdict(list)
        for u in c.split('\n\n'):
            utt = ps.transform_utterance(u, spacy_nlp=spacy_nlp, markers=True)
            for feature, presence in utt.meta['politeness_strategies'].items():
                poli_evolution[feature].append(presence)
        for feature in utt.meta['politeness_strategies'].keys():
            poli_features_mean[feature].append(np.mean(poli_evolution[feature]))   # Calculate the mean

            x = np.arange(len(poli_evolution[feature]))  # X-coordinates (indices of the list)
            y = poli_evolution[feature]                  # Y-coordinates (values of the list)
            try:
                slope, _ = np.polyfit(x, y, 1)       # Fitting a linear regression (polynomial of degree 1) to the data
            except Exception as e:
                print(e)
                slope = 0 
            poli_features_grad[feature].append(slope)    # The slope is the gradient we want

    a = pd.DataFrame(poli_features_mean).add_suffix('_mean')
    b = pd.DataFrame(poli_features_grad).add_suffix('_grad')
    poli_feat_df = pd.concat([a, b], axis=1)

else:
    if len(conversations) == 213:
        poli_feat_df = pd.read_csv('213_wikitac_politeness_markers.csv') 
    if len(conversations) == 542:
        poli_feat_df = pd.read_csv('542_oum_politeness_markers.csv') 
    if len(conversations) == 1000:
        poli_feat_df = pd.read_csv('1000_afd_politeness_markers.csv')

# Detailed codes: https://github.com/CornellNLP/ConvoKit/blob/master/convokit/politeness_collections/politeness_api/features/politeness_strategies.py
feature_names = ['Please', "Start with 'Please'", 'Has Subject Hedge', "Use of 'by the way'",
                 "Hedge words", "Assert factuality", "Start with deference", "Gratitude", "Apologising",
                 "1st person plural", "1st person pronouns", "Start with 1st person",
                 "2nd person pronouns", "Start with 2nd person",
                 "Start with greeting", "Starts with Question", "Starts with Conjunction",
                 "Positive sentiment words", "Negative sentiment words", "Subjunctive words", "Indicative words"]

feature_names = [f + ', x̄' for f in feature_names] +  [f + ', ∇' for f in feature_names]
feature_names = ['PM - ' + f for f in feature_names]
poli_feat_df.columns = feature_names
poli_feat_df

,"PM - Please, x̄","PM - Start with 'Please', x̄","PM - Has Subject Hedge, x̄","PM - Use of 'by the way', x̄","PM - Hedge words, x̄","PM - Assert factuality, x̄","PM - Start with deference, x̄","PM - Gratitude, x̄","PM - Apologising, x̄","PM - 1st person plural, x̄",...,"PM - Start with 1st person, ∇","PM - 2nd person pronouns, ∇","PM - Start with 2nd person, ∇","PM - Start with greeting, ∇","PM - Starts with Question, ∇","PM - Starts with Conjunction, ∇","PM - Positive sentiment words, ∇","PM - Negative sentiment words, ∇","PM - Subjunctive words, ∇","PM - Indicative words, ∇"
0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.250000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000,-0.300000,0.000000,0.0,0.0
1,0.0,0.0,0.272727,0.0,0.090909,0.000000,0.0,0.0,0.000000,0.181818,...,5.891684e-18,0.0,0.000000,0.0,0.0,0.036364,0.063636,-0.063636,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.100000,0.0,0.0,0.000000,0.100000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000,-0.054545,0.030303,0.0,0.0
3,0.0,0.0,0.200000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.200000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000,-0.200000,-0.100000,0.0,0.0
4,0.0,0.0,0.444444,0.0,0.333333,0.111111,0.0,0.0,0.111111,0.000000,...,0.000000e+00,0.0,-0.016667,0.0,0.0,0.000000,-0.066667,-0.016667,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000,0.000000,-0.500000,0.0,0.0
996,0.0,0.0,0.250000,0.0,0.062500,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000e+00,0.0,0.000000,0.0,0.0,0.000000,0.007353,-0.033824,0.0,0.0
997,0.0,0.0,0.400000,0.0,0.200000,0.000000,0.0,0.0,0.000000,0.000000,...,-1.000000e-01,0.0,0.000000,0.0,0.0,0.000000,0.100000,0.000000,0.0,0.0
998,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,8.571429e-02,0.0,0.000000,0.0,0.0,0.000000,0.085714,-0.142857,0.0,0.0


## 1.3. Training the Feature-based Model for the OUM dataset (Regression).
All LLM-generated features and discrete features about collaboration markers (in tandem with the politeness markers above) should have been already generated; see "feature_engineering" folder for details on how we generate those feature sets beyond politeness markers.

### 1.3.1 Reading Data

In [10]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

# X = poli_feat_df  ### Politeness Markers Data
y = pd.DataFrame(data={'labels':labels})['labels']

In [ ]:
if len(conversations) == 542:
    colab_markers_df = pd.read_csv('542_oum_collaboration_markers.csv')  ### Collaboration Markers DATA
if len(conversations) == 213:
    colab_markers_df = pd.read_csv('213_wikitac_collaboration_markers.csv')
if len(conversations) == 1000:
    colab_markers_df = pd.read_csv('1000_afd_collaboration_markers.csv')

feature_names = ['# words', '# me pronoun', '# we pronoun', '# you pronoun', '# 3rd person pronouns',
                 '# Geography terms', '# Meta terms', '# Certainty terms', '# Headging terms',
                 '# New content words', '# New content words * # Certainty terms', '# New content words * # Hedging terms']

feature_names = [f + ', x̄' for f in feature_names] +  [f + ', ∇' for f in feature_names]
feature_names = ['CM - ' + f for f in feature_names]
colab_markers_df.columns = feature_names

In [ ]:
if len(conversations) == 542:
    dispute_tactics_df = pd.read_csv('542_oum_gpt4_dispute_tactics.csv')  ### Dispute Tactics 
if len(conversations) == 213:
    dispute_tactics_df = pd.read_csv('213_wikitac_gpt4_dispute_tactics.csv')
if len(conversations) == 1000:
    dispute_tactics_df = pd.read_csv('1000_afd_gpt4_dispute_tactics.csv')

feature_names = ['Name calling/hostility', 'Ad hominem/ad argument', 'Attempted derailing/off-topic', 'Policing the discussion',
                 'Stating your stance/repeated argument', 'Counterargument', 'Refutation', 'Refuting the central point',
                 'Bailing out', 'Contextualisation', 'Asking questions', 'Providing clarification', 'Suggesting a compromise',
                 'Coordinating', 'Conceding/recanting', 'I don’t know', 'Others non-disagreement tactics']

feature_names = [f + ', x̄' for f in feature_names] +  [f + ', ∇' for f in feature_names]
feature_names = ['DT - ' + f for f in feature_names]
dispute_tactics_df.columns = feature_names

In [12]:
if len(conversations) == 542:
    AQ_feat_df = pd.read_csv('542_oum_gpt4_AQ_1dim.csv') ### Argument Quality 
if len(conversations) == 213:
    AQ_feat_df = pd.read_csv('213_wikitac_gpt4_AQ_1dim.csv')
if len(conversations) == 1000:
    AQ_feat_df = pd.read_csv('1000_afd_gpt4_AQ_1dim.csv')
    aq_mean = AQ_feat_df['AQ'].mean() # row 10 is empty
    AQ_feat_df['AQ'] = AQ_feat_df['AQ'].fillna(aq_mean)
AQ_feat_df['Quality of Arguments'] = AQ_feat_df['AQ'] # Rename
del AQ_feat_df['AQ']

In [14]:
if len(conversations) == 542:
    crlf_df = pd.read_csv('542_oum_gpt4_crlf.csv')  ### Information Content & Styles and Tones
if len(conversations) == 213:
    crlf_df = pd.read_csv('213_wikitac_gpt4_crlf.csv')
if len(conversations) == 1000:
    crlf_df = pd.read_csv('1000_afd_gpt4_crlf.csv')

feature_names = ['Low propositional density', 'High propositional density', 'low Content density', 'high Content density',
                 'Low formality', 'High formality',
                 'Low politeness', 'High politeness',
                 'Negative sentiment', 'Neutral sentiment', 'Positive sentiment',
                 'Epistemic uncertainty', 'Doxastic uncertainty', 'Investigative uncertainty', 'Conditional uncertainty', 'No uncertainty']

feature_names = [f + ', x̄' for f in feature_names] +  [f + ', ∇' for f in feature_names]
feature_names = ['IC - ' + f if 'density' in f else f for f in feature_names]
feature_names = ['ST - ' + f if 'density' not in f else f for f in feature_names]
crlf_df.columns = feature_names

Index(['IC - Low propositional density, x̄',
       'IC - High propositional density, x̄', 'IC - low Content density, x̄',
       'IC - high Content density, x̄', 'ST - Low formality, x̄',
       'ST - High formality, x̄', 'ST - Low politeness, x̄',
       'ST - High politeness, x̄', 'ST - Negative sentiment, x̄',
       'ST - Neutral sentiment, x̄', 'ST - Positive sentiment, x̄',
       'ST - Epistemic uncertainty, x̄', 'ST - Doxastic uncertainty, x̄',
       'ST - Investigative uncertainty, x̄',
       'ST - Conditional uncertainty, x̄', 'ST - No uncertainty, x̄',
       'IC - Low propositional density, ∇',
       'IC - High propositional density, ∇', 'IC - low Content density, ∇',
       'IC - high Content density, ∇', 'ST - Low formality, ∇',
       'ST - High formality, ∇', 'ST - Low politeness, ∇',
       'ST - High politeness, ∇', 'ST - Negative sentiment, ∇',
       'ST - Neutral sentiment, ∇', 'ST - Positive sentiment, ∇',
       'ST - Epistemic uncertainty, ∇', 'ST - Doxastic

In [23]:
N = len(conversations)

In [24]:
X = pd.concat([poli_feat_df.iloc[:N,:],
               colab_markers_df.iloc[:N, :],
               AQ_feat_df.iloc[:N, -1:],
               dispute_tactics_df.iloc[:N, :],
               crlf_df.iloc[:, :] # crlf_df.iloc[:, [0, 1, 2, 3, 16, 17, 18, 19]] --> Information Content; [i for i in range(32) if i not in [0, 1, 2, 3, 16, 17, 18, 19]] --> Style and Tone
               ], axis=1)

constant_columns = X.columns[X.nunique() == 1]
X = X.drop(columns=constant_columns)
if len(constant_columns) > 0:
    print("Removed constant columns:", constant_columns)
else:
    print("No constant columns found in X.")

X = X.reset_index(drop=True)
full_data = pd.concat([X, pd.DataFrame(data={'Target variable':labels})], axis = 1)

No constant columns found in X.


Index(['DT - Name calling/hostility, x̄', 'DT - Ad hominem/ad argument, x̄',
       'DT - Attempted derailing/off-topic, x̄',
       'DT - Policing the discussion, x̄',
       'DT - Stating your stance/repeated argument, x̄',
       'DT - Counterargument, x̄', 'DT - Refutation, x̄',
       'DT - Refuting the central point, x̄', 'DT - Bailing out, x̄',
       'DT - Contextualisation, x̄', 'DT - Asking questions, x̄',
       'DT - Providing clarification, x̄', 'DT - Suggesting a compromise, x̄',
       'DT - Coordinating, x̄', 'DT - Conceding/recanting, x̄',
       'DT - I don’t know, x̄', 'DT - Others non-disagreement tactics, x̄',
       'DT - Name calling/hostility, ∇', 'DT - Ad hominem/ad argument, ∇',
       'DT - Attempted derailing/off-topic, ∇',
       'DT - Policing the discussion, ∇',
       'DT - Stating your stance/repeated argument, ∇',
       'DT - Counterargument, ∇', 'DT - Refutation, ∇',
       'DT - Refuting the central point, ∇', 'DT - Bailing out, ∇',
       'DT - Con

### 1.3.2. Ridge Linear Regression model / Logistic Regression 

#### 1.3.2.1. Ridge Regression

In [ ]:
def conv2topic(c):
  if 'Brexit' in c:
    return 'Brexit'

  elif 'veganism' in c or 'vegan' in c:
    return 'veganism'

  elif 'vaccination' in c or 'vaccine' in c:
    return 'vaccination'

  else:
    raise 'NO TOPIC WAS CHOSEN!'

if len(conversations) == 542:
    topics = [conv2topic(c) for c in conversations]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
import numpy as np
import pandas as pd

# Different seeds for reproducibility
seeds = [42, 123, 456]

# Alpha values to test
alphas = [0.1, 1, 10, 100]

# To store the results for finding the optimal alpha
optimal_results = []

# Step 1: Finding the optimal alpha based on seed=1
print('Finding optimal alpha with seed=1')
kf = KFold(n_splits=7, shuffle=True, random_state=1)

for alpha in alphas:
    print('\nALPHA:', alpha)
    model = Ridge(alpha=alpha)

    all_y_pred_test = []
    all_y_test = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)

        y_pred_test = model.predict(X_test)
        all_y_pred_test.extend(y_pred_test)
        all_y_test.extend(y_test)

    mae_test = mean_absolute_error(all_y_test, all_y_pred_test)
    corr_test, _ = spearmanr(all_y_pred_test, all_y_test)
    mean_score = (mae_test + (1 - corr_test)) / 2  

    optimal_results.append((alpha, mean_score))
    print('MAE:', mae_test)
    print('Spearman Correlation:', corr_test)
    print('Mean Score:', mean_score)

optimal_alpha = min(optimal_results, key=lambda x: x[1])[0]
print(f'\nOptimal alpha found: {optimal_alpha}')

# Step 2: Evaluate the optimal alpha across different seeds and report mean and SD
results = {'mae_test': [], 'corr_test': []}

for seed in seeds:
    print(f'\nEvaluating with seed={seed}')
    kf = KFold(n_splits=7, shuffle=True, random_state=seed)
    model = Ridge(alpha=optimal_alpha, max_iter=10000)

    all_y_pred_test = []
    all_y_test = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)

        y_pred_test = model.predict(X_test)
        all_y_pred_test.extend(y_pred_test)
        all_y_test.extend(y_test)

    mae_test = mean_absolute_error(all_y_test, all_y_pred_test)
    corr_test, _ = spearmanr(all_y_pred_test, all_y_test)

    results['mae_test'].append(mae_test)
    results['corr_test'].append(corr_test)

    print('MAE:', mae_test)
    print('Spearman Correlation:', corr_test)

# Calculate mean and standard deviation for MAE and Spearman correlation
mean_mae_test = np.mean(results['mae_test'])
std_mae_test = np.std(results['mae_test'])
mean_corr_test = np.mean(results['corr_test'])
std_corr_test = np.std(results['corr_test'])

print(f'\nResults for optimal alpha={optimal_alpha}:')
print(f'Mean MAE Test: {mean_mae_test:.3f} ({std_mae_test:.3f})')
print(f'Mean Spearman Correlation Test: {mean_corr_test:.3f} ({std_corr_test:.3f})')



Finding optimal alpha with seed=1

ALPHA: 0.1
MAE: 1.4702449850389454
Spearman Correlation: 0.34619843919521837
Mean Score: 1.0620232729218635

ALPHA: 1
MAE: 1.4731830445589564
Spearman Correlation: 0.33350949810580227
Mean Score: 1.069836773226577

ALPHA: 10
MAE: 1.513405180056012
Spearman Correlation: 0.3146294194720946
Mean Score: 1.0993878802919586

ALPHA: 100
MAE: 1.59368241797387
Spearman Correlation: 0.22172861349417122
Mean Score: 1.1859769022398494

Optimal alpha found: 0.1

Evaluating with seed=42
MAE: 1.4593720820279288
Spearman Correlation: 0.3532750676518215

Evaluating with seed=123
MAE: 1.4643285954159402
Spearman Correlation: 0.3548715651200828

Evaluating with seed=456
MAE: 1.4799838190812995
Spearman Correlation: 0.33581348645748044

Results for optimal alpha=0.1:
Mean MAE Test: 1.468 (0.009)
Mean Spearman Correlation Test: 0.348 (0.009)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr

# Function to calculate mean and standard deviation
def calculate_mean_sd(values):
    return np.mean(values), np.std(values)

# Initialize dictionaries to store the results
results = {
    'Brexit': {'mae': [], 'corr': []},
    'vaccination': {'mae': [], 'corr': []},
    'veganism': {'mae': [], 'corr': []}
}

for seed in [42, 123, 456]:
    # Initialize KFold (we use leave-one-out approach)
    kf = KFold(n_splits=7, shuffle=True, random_state=seed)

    # Initialize the Ridge regression model with an alpha value
    model = Ridge(alpha=1.0)

    # To accumulate predictions and true values
    all_y_pred_train = []
    all_y_train = []
    all_y_pred_test = []
    all_y_test = []

    for train_index, test_index in kf.split(X):
        # Split into training and testing sets using .iloc for pandas
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit the model on the training data
        model.fit(X_train, y_train)

        # Training Predictions
        y_pred_train = model.predict(X_train)
        all_y_pred_train.extend(y_pred_train)
        all_y_train.extend(y_train)

        # Testing Predictions
        y_pred_test = model.predict(X_test)
        if len(X) == 542:
            y_pred_test = np.clip(y_pred_test, 1, 7)

        all_y_pred_test.extend(y_pred_test)
        all_y_test.extend(y_test)

    # Evaluate TEST
    mae_test = mean_absolute_error(all_y_test, all_y_pred_test)
    corr_test, _ = spearmanr(all_y_pred_test, all_y_test)
    print('\n\n\n\nTEST:')
    print("Mean Absolute Error (MAE):", mae_test)
    print("Spearman Correlation:", corr_test)

    if len(X) == 542:
        # Convert topics to a numpy array for efficient indexing
        topics_array = np.array(topics)

        # Unique topics for iteration
        unique_topics = ['Brexit', 'vaccination', 'veganism']

        for topic in unique_topics:
            # Identify indexes for the current topic
            topic_indexes_test = np.where(topics_array == topic)[0]  # For test data
            topic_indexes_train = np.where(topics_array == topic)[0]  # For train data

            # Filter predictions and true values based on the topic
            y_pred_train_topic = np.array(all_y_pred_train)[topic_indexes_train]
            y_train_topic = np.array(all_y_train)[topic_indexes_train]
            y_pred_test_topic = np.array(all_y_pred_test)[topic_indexes_test]
            y_test_topic = np.array(all_y_test)[topic_indexes_test]

            # Calculate and print metrics for the current topic - TEST
            mae_test_topic = mean_absolute_error(y_test_topic, y_pred_test_topic)
            corr_test_topic, _ = spearmanr(y_pred_test_topic, y_test_topic)
            print(f'\nTEST - {topic}:')
            print(f"Mean Absolute Error (MAE): {mae_test_topic}")
            print(f"Spearman Correlation: {corr_test_topic}")
            print("\n")  # For better separation in output

            # Store the results
            results[topic]['mae'].append(mae_test_topic)
            results[topic]['corr'].append(corr_test_topic)

# Calculate mean and SD for each topic
for topic in results:
    mae_mean, mae_sd = calculate_mean_sd(results[topic]['mae'])
    corr_mean, corr_sd = calculate_mean_sd(results[topic]['corr'])
    print(f'{topic} - MAE Mean: {round(mae_mean,3)}({round(mae_sd,3)})')
    print(f'{topic} - Correlation Mean: {round(corr_mean,3)} ({round(corr_sd,3)})')





TEST:
Mean Absolute Error (MAE): 1.4628048820059367
Spearman Correlation: 0.34296528152139055

TEST - Brexit:
Mean Absolute Error (MAE): 1.3823092521502698
Spearman Correlation: 0.4606229905032811



TEST - vaccination:
Mean Absolute Error (MAE): 1.4718787399484359
Spearman Correlation: 0.337556617484527



TEST - veganism:
Mean Absolute Error (MAE): 1.5131915962086917
Spearman Correlation: 0.2519040930195204






TEST:
Mean Absolute Error (MAE): 1.470847255417941
Spearman Correlation: 0.34228316462234165

TEST - Brexit:
Mean Absolute Error (MAE): 1.551764511217121
Spearman Correlation: 0.34454990674098507



TEST - vaccination:
Mean Absolute Error (MAE): 1.383218223303109
Spearman Correlation: 0.43167746893831693



TEST - veganism:
Mean Absolute Error (MAE): 1.4929316473572813
Spearman Correlation: 0.25318642627490795






TEST:
Mean Absolute Error (MAE): 1.475594359226507
Spearman Correlation: 0.33301438686768214

TEST - Brexit:
Mean Absolute Error (MAE): 1.427129412060035
Spe

#### 1.3.2.2. Logistic Regression

In [25]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, recall_score, precision_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


best_hyperparams = {'C': None, 'solver': None}
best_score = 0

for C in [0.1, 1, 10, 100]:
    for solver in {'lbfgs', 'liblinear', 'sag', 'saga'}:
        print('\n\n', C, solver)
        # Initialize KFold (we use leave-one-out approach)
        kf = KFold(n_splits=7, shuffle=True, random_state=1)

        # Initialize the Logistic Regression model
        model = LogisticRegression(random_state=42, max_iter=10000, C=C, solver=solver)

        # To accumulate predictions and true values
        all_y_pred_train = []
        all_y_train = []
        all_y_pred_test = []
        all_y_test = []

        for train_index, test_index in kf.split(X):
            # Split into training and testing sets using .iloc for pandas
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Fit the model on the training data
            model.fit(X_train, y_train)

            # Training Predictions
            y_pred_train = model.predict_proba(X_train)[:, 1]
            all_y_pred_train.extend(y_pred_train)
            all_y_train.extend(y_train)

            # Testing Predictions
            y_pred_test = model.predict_proba(X_test)[:, 1]
            all_y_pred_test.extend(y_pred_test)
            all_y_test.extend(y_test)

        threshold = Counter(y_train)[1] / len(y_train)

        # Evaluate TEST
        auroc_test = roc_auc_score(all_y_test, all_y_pred_test)
        precision_test, recall_test, _ = precision_recall_curve(all_y_test, all_y_pred_test)
        auprc_test = auc(recall_test, precision_test)

        # Calculate recall, precision, and specificity
        y_pred_test_binary = [1 if p >= threshold else 0 for p in all_y_pred_test]
        recall = recall_score(all_y_test, y_pred_test_binary)
        precision = precision_score(all_y_test, y_pred_test_binary)
        cm_test = confusion_matrix(all_y_test, y_pred_test_binary)
        tn, fp, fn, tp = cm_test.ravel()
        specificity = tn / (tn + fp)

        print('TEST:')
        print("AUROC:", auroc_test)
        print("AUPRC:", auprc_test)
        print(cm_test)

        # Calculate the mean of AUROC and AUPRC
        mean_score = (auroc_test + auprc_test) / 2

        # Update best hyperparameters if current mean score is higher
        if mean_score > best_score:
            best_score = mean_score
            best_hyperparams['C'] = C
            best_hyperparams['solver'] = solver

print(f'Best hyperparameters: C={best_hyperparams["C"]}, solver={best_hyperparams["solver"]}')
print(f'Best mean score (AUROC + AUPRC) / 2: {best_score}')




 0.1 lbfgs
TEST:
AUROC: 0.7564361143906599
AUPRC: 0.4445065400024353
[[512 244]
 [ 65 177]]


 0.1 saga
TEST:
AUROC: 0.7564197166469894
AUPRC: 0.4444875483697387
[[512 244]
 [ 65 177]]


 0.1 sag
TEST:
AUROC: 0.7564306484761031
AUPRC: 0.4444903944473114
[[512 244]
 [ 65 177]]


 0.1 liblinear
TEST:
AUROC: 0.7515331890331891
AUPRC: 0.4446118594697436
[[492 264]
 [ 58 184]]


 1 lbfgs
TEST:
AUROC: 0.7651159867068957
AUPRC: 0.45872020917106227
[[539 217]
 [ 77 165]]


 1 saga
TEST:
AUROC: 0.7651159867068958
AUPRC: 0.4587210337036245
[[539 217]
 [ 77 165]]


 1 sag
TEST:
AUROC: 0.7651214526214526
AUPRC: 0.45872975827707624
[[539 217]
 [ 77 165]]


 1 liblinear
TEST:
AUROC: 0.7646349862258953
AUPRC: 0.4590650287995283
[[539 217]
 [ 77 165]]


 10 lbfgs
TEST:
AUROC: 0.7662474310201584
AUPRC: 0.46558921503109657
[[552 204]
 [ 73 169]]


 10 saga
TEST:
AUROC: 0.7662747605929423
AUPRC: 0.465602352727299
[[552 204]
 [ 73 169]]


 10 sag
TEST:
AUROC: 0.7662583628492718
AUPRC: 0.4655807628608092

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, recall_score, precision_score, confusion_matrix
import numpy as np
from collections import Counter

# Initialize the optimal hyperparameters
C = best_hyperparams["C"]
solver = best_hyperparams["solver"]
seeds = [42, 123, 456]

# To accumulate the results for each seed
auroc_scores = []
auprc_scores = []
recall_scores = []
precision_scores = []
specificity_scores = []

for seed in seeds:
    print(f'\n\nRunning for seed {seed}')

    # Initialize KFold (we use leave-one-out approach)
    kf = KFold(n_splits=7, shuffle=True, random_state=seed)

    # Initialize the Logistic Regression model
    model = LogisticRegression(random_state=seed, max_iter=5000, C=C, solver=solver)

    # To accumulate predictions and true values
    all_y_pred_train = []
    all_y_train = []
    all_y_pred_test = []
    all_y_test = []

    for train_index, test_index in kf.split(X):
        # Split into training and testing sets using .iloc for pandas
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit the model on the training data
        model.fit(X_train, y_train)

        # Training Predictions
        y_pred_train = model.predict_proba(X_train)[:, 1]
        all_y_pred_train.extend(y_pred_train)
        all_y_train.extend(y_train)

        # Testing Predictions
        y_pred_test = model.predict_proba(X_test)[:, 1]
        all_y_pred_test.extend(y_pred_test)
        all_y_test.extend(y_test)

    threshold = Counter(y_train)[1] / len(y_train)

    # Evaluate TEST
    auroc_test = roc_auc_score(all_y_test, all_y_pred_test)
    precision_test, recall_test, _ = precision_recall_curve(all_y_test, all_y_pred_test)
    auprc_test = auc(recall_test, precision_test)

    # Store the results
    auroc_scores.append(auroc_test)
    auprc_scores.append(auprc_test)

# Calculate mean and standard deviation for each metric
auroc_mean = np.mean(auroc_scores)
auroc_sd = np.std(auroc_scores)
auprc_mean = np.mean(auprc_scores)
auprc_sd = np.std(auprc_scores)

# Print the results
print('\n\nTEST:')
print(f"AUROC: {auroc_mean:.3f} ({auroc_sd:.3f})")
print(f"AUPRC: {auprc_mean:.3f} ({auprc_sd:.3f})")



Running for seed 42


Running for seed 123


Running for seed 456


TEST:
AUROC: 0.918 (0.000)
AUPRC: 0.772 (0.002)
